<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/ResNet_cifar100_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
cifar100 = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

print("x shape")
print(f"train: {x_train.shape}, val: {x_val.shape}, test: {x_test.shape}")
print("y shape")
print(f"train: {y_train.shape}, val: {y_val.shape}, test: {y_test.shape}")


trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
trainloader = trainloader.shuffle(1024).batch(128).prefetch(tf.data.experimental.AUTOTUNE)
valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
valloader = valloader.batch(128).prefetch(tf.data.experimental.AUTOTUNE)
testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
testloader = testloader.batch(128).prefetch(tf.data.experimental.AUTOTUNE)

169009152/169001437 [==============================] - 4s 0us/step
x shape
train: (40000, 32, 32, 3), val: (10000, 32, 32, 3), test: (10000, 32, 32, 3)
y shape
train: (40000, 1), val: (10000, 1), test: (10000, 1)


In [3]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(100, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    return model

In [4]:
net = create_res_net()
#net = Model()
net.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

result = net.fit(
    trainloader, 
    epochs = 5,
    validation_data=valloader)

keras.backend.clear_session() 

Epoch 1/5
313/313 [==============================] - 51s 104ms/step - loss: 3.9209 - accuracy: 0.1000 - val_loss: 5.2235 - val_accuracy: 0.0500
Epoch 2/5
313/313 [==============================] - 32s 102ms/step - loss: 3.3609 - accuracy: 0.1871 - val_loss: 4.0864 - val_accuracy: 0.1563
Epoch 3/5
313/313 [==============================] - 32s 103ms/step - loss: 2.9123 - accuracy: 0.2693 - val_loss: 3.2225 - val_accuracy: 0.2479
Epoch 4/5
313/313 [==============================] - 32s 102ms/step - loss: 2.5084 - accuracy: 0.3439 - val_loss: 2.6613 - val_accuracy: 0.3205
Epoch 5/5
313/313 [==============================] - 32s 103ms/step - loss: 2.1335 - accuracy: 0.4267 - val_loss: 2.3711 - val_accuracy: 0.3833


NameError: ignored